# Voice command bot
Source: https://www.analyticsvidhya.com/blog/2021/10/complete-guide-to-build-your-ai-chatbot-with-nlp-in-python/

In [ ]:
#preload data of helpline numbers and resources


In [ ]:
# for speech-to-text
import speech_recognition as sr
# for text-to-speech
from gtts import gTTS
# for language model
import transformers
import os
import time
# for data
import os
import datetime
import numpy as np
# Building the AI
class ChatBot():
    def __init__(self, name):
        print("----- Starting up", name, "-----")
        self.name = name
    def speech_to_text(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            print("Listening...")
            audio = recognizer.listen(mic)
            self.text="ERROR"
        try:
            self.text = recognizer.recognize_google(audio)
            print("Me  --> ", self.text)
        except:
            print("Me  -->  ERROR")
    @staticmethod
    def text_to_speech(text):
        print("Fern --> ", text)
        speaker = gTTS(text=text, lang="en", slow=False)
        speaker.save("res.mp3")
        statbuf = os.stat("res.mp3")
        mbytes = statbuf.st_size / 1024
        duration = mbytes / 200
        os.system('start res.mp3') 
        # os.system("close res.mp3")
        time.sleep(int(50*duration))
        os.remove("res.mp3")
    def wake_up(self, text):
        return True if self.name in text.lower() else False
    @staticmethod
    def action_time():
        return datetime.datetime.now().time().strftime('%H:%M')
# Running the AI
if __name__ == "__main__":
    ai = ChatBot(name="Fern")
    nlp = transformers.pipeline("conversational", model="microsoft/DialoGPT-medium")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    ex=True
    while ex:
        ai.speech_to_text()
        ## wake up
        if ai.wake_up(ai.text) is True:
            res = "Hello, my name is Fern and I will be assisting you today. I am a chatbot that aims to help victims of domestic violence by offering solutions and recommending lawyers. I will begin with asking you a range of questions to better understand your current situation. Based on the information you have provided, I will give some recommendations on how you can move forward. In case you would like further assistance by a lawyer, we will also provide a list of lawyers that we think are best suited for your needs. Before we delve in, please know that the more details you can provide us, the better advice we will be able to give you. These small details can be very important from a legal perspective so please be truthful so we can best assess your legal position. We understand that this is a sensitive matter so please take as much time as you need and ensure that you are in a safe environment to talk to us. If you are living with the person who is inflicting harm on you, you can click the red button at the top right to disguise this website as an online shopping site. Does everything make sense so far?"
            ai.text_to_speech(res)
            ai.speech_to_text()
            if any(i in ai.text for i in ["yes","ok","okay"]):
                res="To begin with, everything we discuss today is bound by strict confidentiality and no information will be disclosed unless we have your authority or are permitted or compelled to do so by law. In exceptional circumstances, we would disclose information to prevent crime or imminent serious physical harm being carried out. May I begin the questions?"
                ai.text_to_speech(res)
                ai.speech_to_text()
                if any(i in ai.text for i in ["yes","ok","okay"]):
                    if any(i in ai.text for i in ["aggressive","unsafe","uncomfortable","danger","violent"]):
                        res = np.random.choice(["what kind of domestic issues are you experiencing?"])
                        print(res)
                        ai.speech_to_text()
                        if any(i in ai.text for i in ["physical","hitting me","beating me","broke my nose"]):#physical abuse
                            res = np.random.choice(["Can you describe whether you are being threatened or harmed?"])
                            ai.text_to_speech(res)
                            if any(i in ai.text for i in ["yes"]):
                                res=np.random(["Has there been any evidence of property damage?"])
                                ai.text_to_speech(res)
                                ai.speech_to_text()
                                print("Here are some resources you can consult to assist you with your situation.")
                                ex=False
                            elif any(i in ai.text for i in ["no"]):
                                res=np.random(["Can you describe in more detail what your partner has been doing to physically harm you?"])
                                ai.text_to_speech(res)
                                ai.speech_to_text()
                                print("Here are some resources you can consult to assist you with your situation.")
                                ex=False
                            else:
                                while not any(i in ai.text for i in ["domestic","unsafe","uncomfortable","danger","physical","hitting me","beating me","broke my nose"]):
                                    res=np.random.choice(["Sorry I didn't understand that response"])
                                    ai.text_to_speech(res)
                                    ai.speech_to_text()
                        elif any(i in ai.text for i in ["gaslight","depression","anxiety","psychological"]):#emotional abuse
                            res = np.random.choice(["Here are some resources to access the assistance you require. In the meantime, this chat transcript will automatically save and be sent to our legal aid to create a case for you. Thank you for using our services. "])
                            ai.text_to_speech(res)
                            ai.speech_to_text()
                        elif any(i in ai.text for i in ["grope","rape","no consent","unconsented","violated","pregnant","penetrated","drugged"]):#sexual abuse
                            res = np.random.choice(["Here are some resources to access the assistance you require. In the meantime, this chat transcript will automatically save and be sent to our legal aid to create a case for you. Thank you for using our services. "])
                            ai.text_to_speech(res)
                            ai.speech_to_text()
                        elif any(i in ai.text for i in ["money","bank account","credit card","debit card","financial","stealing my money"]): #financial abuse
                            res = np.random.choice(["Here are some resources to access the assistance you require. In the meantime, this chat transcript will automatically save and be sent to our legal aid to create a case for you. Thank you for using our services. "])
                            ai.text_to_speech(res)
                            ai.speech_to_text()
                        else:
                            while not any(i in ai.text for i in ["domestic","unsafe","uncomfortable","danger","physical","hitting me","beating me","broke my nose",
                                                                 "grope","rape","no consent","unconsented","violated","pregnant","penetrated","drugged",
                                                                "money","bank account","credit card","debit card","financial","stealing my money"]):
                                res=np.random.choice(["Sorry I didn't understand that response"])
                                ai.text_to_speech(res)
                                ai.speech_to_text()
                    elif any(i in ai.text for i in ["bye"]):
                        res = np.random.choice(["If you have an emergency, be sure to call 000 immediately"])
                        ai.text_to_speech(res)
                        ex=False
                    else:   
                        if ai.text=="ERROR":
                            res="can you describe your situation more clearly?"
                        else:
                            chat = nlp(transformers.Conversation(ai.text), pad_token_id=50256)
                            res = str(chat)
                            res = res[res.find("Fern >> ")+6:].strip()
                    ai.text_to_speech(res)
            else:
                ex=False#no agreeing to t&c
print("This chat transcript will automatically save and be sent to our legal aid to create a case for you. Thank you for using our services. ")
    